# Getting Data From Oracle

In [1]:
import mlrun
project = mlrun.get_or_create_project("oracle-npra", "./", user_project=True)

> 2023-09-28 14:08:07,407 [info] Loading project from path: {'project_name': 'oracle-npra', 'path': './'}
> 2023-09-28 14:08:07,782 [info] Project loaded successfully: {'project_name': 'oracle-npra', 'path': './', 'stored_in_db': True}


In [2]:
!pip install oracledb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 6.1 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import oracledb
import pandas as pd

username = "C##harish"
password = "harish1438"
host = "192.168.31.75"
port = "1521"
service_name = "xe"
    
conn = oracledb.connect(user=username, password=password, host=host, port=port, service_name=service_name)
    
# Create a cursor
cursor = conn.cursor()

# Define the SQL statement for selecting data
select_sql = "SELECT * FROM NPRA_IMPORT"

# Execute the SQL statement to fetch data
cursor.execute(select_sql)

# Fetch all rows of data
rows = cursor.fetchall()
    
# Define the SQL statement to fetch column names for a table
table_name = 'NPRA_IMPORT'
select_sql2 = f"SELECT column_name FROM all_tab_columns WHERE table_name = '{table_name}'"

# Execute the SQL statement to fetch data
cursor.execute(select_sql2)

# Fetch all rows of data
columns = cursor.fetchall()
        
# Define column names
column_names = [column[0].lower() for column in columns]

# Create a DataFrame from the fetched rows with column names
df = pd.DataFrame(rows, columns=column_names)

df.head()

DatabaseError: ORA-00942: table or view does not exist
Help: https://docs.oracle.com/error-help/db/ora-00942/

In [8]:
%%writefile data-oracle.py

import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest, f_classif
import oracledb

import mlrun

def get_oracle_data():
    # Connection details
    username = "C##harish"
    password = "harish1438"
    host = "192.168.31.75"
    port = "1521"
    service_name = "xe"
    
    conn = oracledb.connect(user=username, password=password, host=host, port=port, service_name=service_name)
    
    # Create a cursor
    cursor = conn.cursor()

    # Define the SQL statement for selecting data
    select_sql = "SELECT * FROM NPRA_IMPORT"

    # Execute the SQL statement to fetch data
    cursor.execute(select_sql)

    # Fetch all rows of data
    rows = cursor.fetchall()
    
    # Define the SQL statement to fetch column names for a table
    table_name = 'NPRA_IMPORT'
    select_sql2 = f"SELECT column_name FROM all_tab_columns WHERE table_name = '{table_name}'"

    # Execute the SQL statement to fetch data
    cursor.execute(select_sql2)

    # Fetch all rows of data
    columns = cursor.fetchall()
        
    # Define column names
    column_names = [column[0].lower() for column in columns]

    # Create a DataFrame from the fetched rows with column names
    df = pd.DataFrame(rows, columns=column_names)

    return df

def load_data(df):
    df.drop("customer_id", axis=1, inplace=True)
    df.drop_duplicates(inplace=True)
    df.dropna(axis=0, inplace=True)
    return df


def remove_outliers_iqr(df, col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    df = df[(df[col] >= lower_bound)] 
    df = df[(df[col] <= upper_bound)]
    return df


def label_encode_categorical_columns(df):
    le = LabelEncoder()
    for col in df.select_dtypes(include='object'):
        df[col] = le.fit_transform(df[col])
    return df


def preprocess_data(data):
    can_have_outlier = ["person_age","person_income","person_emp_length","cb_person_cred_hist_length","loan_amnt"]
    for col in can_have_outlier:
        data = remove_outliers_iqr(data, col)
    preprocessed_data = label_encode_categorical_columns(data) 
    return preprocessed_data


def data_balance(new_df):
    # Divide by class
    df_class_0 = new_df[new_df['loan_status'] == 0]
    df_class_1 = new_df[new_df['loan_status'] == 1]
    count_class_0, count_class_1 = new_df['loan_status'].value_counts()
    # random over sampling
    df_class_1_over = df_class_1.sample(count_class_0, replace=True)
    new_df = pd.concat([df_class_0, df_class_1_over], axis=0)
    return new_df


@mlrun.handler(outputs=["dataset", "label_column"])
def credit_risk_dataset_generator():
    """
    A function which generates the credit risk dataset
    """
    dataset = get_oracle_data()
    data = load_data(dataset)
    preprocessed_data = preprocess_data(data)
    for_model_df = data_balance(preprocessed_data)

    return for_model_df, "loan_status"


Writing data-prep-oracle-db.py


In [9]:
project.set_function(
    "data-prep-oracle-db.py",
    name="data-prep2",
    kind="job",
    image="mlrun/mlrun",
    handler="credit_risk_dataset_generator",
    requirements=["mlrun[complete]", "scikit-learn~=1.3.0", "oracledb"],
)

> 2023-09-28 10:17:18,819 [warning] Failed to add git metadata, ignore if path is not part of a git repo.: {'path': './', 'error': '/home/jovyan/work'}


In [10]:
gen_data_run = project.run_function("data-prep2", local=True)

> 2023-09-28 10:17:24,476 [warning] it is recommended to use k8s secret (specify secret_name), specifying the aws_access_key/aws_secret_key directly is unsafe
> 2023-09-28 10:17:24,502 [info] Storing function: {'name': 'data-prep2-credit-risk-dataset-generator', 'uid': '60e9f12df5844a48ad4ebcfe6f50ae9e', 'db': 'http://mlrun-api:8080'}


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
oraclenpra,...6f50ae9e,0,Sep 28 10:17:24,completed,data-prep2-credit-risk-dataset-generator,kind=owner=jovyanhost=mlrun-jupyter-66659dc6c9-zdvjv,,,label_column=loan_status,dataset


> 2023-09-28 10:17:27,547 [info] Run execution finished: {'status': 'completed', 'name': 'data-prep2-credit-risk-dataset-generator'}


In [11]:
%%writefile trainer-oracle-db.py

import pandas as pd

from sklearn import ensemble
from sklearn.model_selection import train_test_split

import mlrun
from mlrun.frameworks.sklearn import apply_mlrun


@mlrun.handler()
def train(
    dataset: pd.DataFrame,
    label_column: str = "loan_status",
    n_estimators: int = 100,
    learning_rate: float = 0.1,
    max_depth: int = 3,
    model_name: str = "loan_status_classifier",
):
    # Initialize the x & y data
    x = dataset.drop(label_column, axis=1)
    y = dataset[label_column]

    # Train/Test split the dataset
    x_train, x_test, y_train, y_test = train_test_split(
        x, y, test_size=0.2, random_state=42
    )

    # Pick an ideal ML model
    model = ensemble.GradientBoostingClassifier(
        n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth
    )

    # -------------------- The only line you need to add for MLOps -------------------------
    # Wraps the model with MLOps (test set is provided for analysis & accuracy measurements)
    apply_mlrun(model=model, model_name=model_name, x_test=x_test, y_test=y_test)
    # --------------------------------------------------------------------------------------

    # Train the model
    model.fit(x_train, y_train)

Writing trainer-oracle-db.py


In [12]:
trainer = project.set_function(
    "trainer-oracle-db.py", name="trainer2", kind="job", image="mlrun/mlrun", handler="train"
)

> 2023-09-28 10:17:34,666 [warning] Failed to add git metadata, ignore if path is not part of a git repo.: {'path': './', 'error': '/home/jovyan/work'}


In [13]:
hp_tuning_run = project.run_function(
    "trainer2",
    inputs={"dataset": gen_data_run.outputs['dataset']},
    hyperparams={
        "n_estimators": [10, 100, 1000],
        "learning_rate": [1e-1, 1e-3],
        "max_depth": [2, 8],
    },
    selector="max.accuracy",
)

> 2023-09-28 10:17:35,755 [warning] it is recommended to use k8s secret (specify secret_name), specifying the aws_access_key/aws_secret_key directly is unsafe
> 2023-09-28 10:17:35,769 [info] Storing function: {'name': 'trainer2-train', 'uid': '493d50eef02244608df2b24e1bcf8ba7', 'db': 'http://mlrun-api:8080'}
> 2023-09-28 10:17:35,990 [info] Job is running in the background, pod: trainer2-train-88skh
> 2023-09-28 10:22:13,745 [info] best iteration=9, used criteria max.accuracy
> 2023-09-28 10:22:14,583 [info] Run execution finished: {'status': 'completed', 'name': 'trainer2-train'}
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
oraclenpra,...1bcf8ba7,0,Sep 28 10:17:42,completed,trainer2-train,kind=jobowner=jovyanmlrun/client_version=1.4.0mlrun/client_python_version=3.9.13,dataset,,best_iteration=9accuracy=0.9826863249019343f1_score=0.9826980264936469precision_score=0.9805772862152684recall_score=0.9848279599024654,feature-importancetest_setconfusion-matrixroc-curvescalibration-curvemodeliteration_resultsparallel_coordinates


> 2023-09-28 10:22:18,625 [info] Run execution finished: {'status': 'completed', 'name': 'trainer2-train'}


In [19]:
hp_tuning_run.outputs

{'best_iteration': 9,
 'accuracy': 0.9826863249019343,
 'f1_score': 0.9826980264936469,
 'precision_score': 0.9805772862152684,
 'recall_score': 0.9848279599024654,
 'feature-importance': 's3://mlrun/projects/oraclenpra/artifacts/trainer2-train/9/feature-importance.html',
 'test_set': 'store://artifacts/oraclenpra/trainer2-train_test_set:493d50eef02244608df2b24e1bcf8ba7',
 'confusion-matrix': 's3://mlrun/projects/oraclenpra/artifacts/trainer2-train/9/confusion-matrix.html',
 'roc-curves': 's3://mlrun/projects/oraclenpra/artifacts/trainer2-train/9/roc-curves.html',
 'calibration-curve': 's3://mlrun/projects/oraclenpra/artifacts/trainer2-train/9/calibration-curve.html',
 'model': 'store://artifacts/oraclenpra/loan_status_classifier:493d50eef02244608df2b24e1bcf8ba7',
 'iteration_results': 's3://mlrun/projects/oraclenpra/artifacts/trainer2-train/0/iteration_results.csv',
 'parallel_coordinates': 'store://artifacts/oraclenpra/trainer2-train_parallel_coordinates:493d50eef02244608df2b24e1bcf8

In [14]:
# Save the project in the db (and into the project.yaml file)
project.save()

In [15]:
# Deploy the trained model as a serverless function
serving_fn = mlrun.new_function("serving", image="mlrun/mlrun", kind="serving")
serving_fn.with_code(body=" ")
mlrun.deploy_function(
    serving_fn,
    models=[
        {
            "key": "credit-classifier",
            "model_path": hp_tuning_run.outputs["model"],
            "class_name": 'mlrun.frameworks.sklearn.SklearnModelServer',
        }
    ],
)

> 2023-09-28 10:22:18,720 [warning] it is recommended to use k8s secret (specify secret_name), specifying the aws_access_key/aws_secret_key directly is unsafe
> 2023-09-28 10:22:18,727 [info] Starting remote function deploy
2023-09-28 10:22:18  (info) Deploying function
2023-09-28 10:22:18  (info) Building
2023-09-28 10:22:19  (info) Staging files and preparing base images
2023-09-28 10:22:19  (info) Building processor image
2023-09-28 10:29:55  (info) Build complete
2023-09-28 10:30:26  (info) Function deploy complete
> 2023-09-28 10:30:33,145 [info] successfully deployed function: {'internal_invocation_urls': ['nuclio-oraclenpra-serving.mlrun.svc.cluster.local:8080'], 'external_invocation_urls': ['192.168.1.6:30162']}


DeployStatus(state=ready, outputs={'endpoint': 'http://192.168.1.6:30162', 'name': 'oraclenpra-serving'})

In [16]:
# load the project in the current context dir
project = mlrun.load_project("./")

In [21]:
%%writefile './workflow-oracle.py'

from kfp import dsl
import mlrun

# Create a Kubeflow Pipelines pipeline
@dsl.pipeline(name="npra3-demo")
def pipeline(model_name):
    # Run the ingestion function with the new image and params
    ingest = mlrun.run_function(
        "data-prep2",
        name="get-oracle-data",
        outputs=["dataset"],
    )

    # Train a model
    train = mlrun.run_function(
        "trainer2",
        inputs={"dataset": ingest.outputs["dataset"]},
        hyperparams={
            "n_estimators": [10, 100, 1000],
            "learning_rate": [1e-1, 1e-3],
            "max_depth": [2, 8],
        },
        selector="max.accuracy",
        outputs=["model"],
    )

    # Deploy the trained model as a serverless function
    serving_fn = mlrun.new_function("serving", image="mlrun/mlrun", kind="serving")
    serving_fn.with_code(body=" ")
    mlrun.deploy_function(
        serving_fn,
        models=[
            {
                "key": model_name,
                "model_path": train.outputs["model"],
                "class_name": 'mlrun.frameworks.sklearn.SklearnModelServer',
            }
        ],
    )

Overwriting ./workflow-oracle.py


In [22]:
# Run the workflow
run_id = project.run(
    workflow_path="./workflow-oracle.py",
    arguments={"model_name": "credit-classifier","dataset": df},
    watch=True,
)

> 2023-09-28 10:44:18,218 [warning] it is recommended to use k8s secret (specify secret_name), specifying the aws_access_key/aws_secret_key directly is unsafe
> 2023-09-28 10:44:18,345 [warning] POST http://mlrun-api:8080/api/v1/projects/oraclenpra/pipelines request failed, http retries disabled for POST method.: {'exception_type': <class 'requests.exceptions.InvalidHeader'>, 'exception_message': 'Invalid leading whitespace, reserved character(s), or returncharacter(s) in header value: "{\'model_name\': \'credit-classifier\', \'dataset\':        customer_id  person_age  person_income person_home_ownership  \\\\\\n0                1          22          59000                  RENT   \\n1                2          21           9600                   OWN   \\n2                3          25           9600              MORTGAGE   \\n3                4          23          65500                  RENT   \\n4                5          24          54400                  RENT   \\n...           

OSError: error: cannot cannot submit pipeline, Invalid leading whitespace, reserved character(s), or returncharacter(s) in header value: "{'model_name': 'credit-classifier', 'dataset':        customer_id  person_age  person_income person_home_ownership  \\\n0                1          22          59000                  RENT   \n1                2          21           9600                   OWN   \n2                3          25           9600              MORTGAGE   \n3                4          23          65500                  RENT   \n4                5          24          54400                  RENT   \n...            ...         ...            ...                   ...   \n32576        32577          57          53000              MORTGAGE   \n32577        32578          54         120000              MORTGAGE   \n32578        32579          65          76000                  RENT   \n32579        32580          56         150000              MORTGAGE   \n32580        32581    